In [0]:
from pyspark.sql import functions as F
from pyspark.sql.types import *

In [0]:
spark
sc = spark.sparkContext

steam_rdd = sc.textFile("s3://full-stack-bigdata-datasets/Big_Data/Project_Steam/steam_game_output.json")

In [0]:
type(steam_rdd)

Out[17]: pyspark.rdd.RDD

In [0]:
steam_json = spark.read.json("s3://full-stack-bigdata-datasets/Big_Data/Project_Steam/steam_game_output.json")
type(steam_json)

Out[18]: pyspark.sql.dataframe.DataFrame

In [0]:
schema = StructType([
  StructField('data', ArrayType(
        StructType([
            StructField('appid', LongType(), True),
            StructField('categories', ArrayType(StringType(), True), True),
            StructField('ccu', LongType(), True),
        ])
    ), True),
    StructField('id', StringType(), True)
])

In [0]:
users_df = spark.createDataFrame(steam_rdd, schema=schema)
users_df.printSchema()
users_df.show(1)

com.databricks.backend.daemon.driver.DriverClientDestroyedException: abort: DriverClient destroyed
	at com.databricks.backend.daemon.driver.DriverClient.$anonfun$poll$4(DriverClient.scala:754)
	at scala.concurrent.Future.$anonfun$flatMap$1(Future.scala:307)
	at scala.concurrent.impl.Promise.$anonfun$transformWith$1(Promise.scala:54)
	at scala.concurrent.impl.CallbackRunnable.run(Promise.scala:77)
	at com.databricks.threading.ContextBoundRunnable.$anonfun$run$2(ContextBoundRunnable.scala:15)
	at scala.runtime.java8.JFunction0$mcV$sp.apply(JFunction0$mcV$sp.java:23)
	at com.databricks.logging.AttributionContextTracing.$anonfun$withAttributionContext$1(AttributionContextTracing.scala:48)
	at com.databricks.logging.AttributionContext$.$anonfun$withValue$1(AttributionContext.scala:271)
	at scala.util.DynamicVariable.withValue(DynamicVariable.scala:62)
	at com.databricks.logging.AttributionContext$.withValue(AttributionContext.scala:267)
	at com.databricks.logging.AttributionContextTracing.withAttributionContext(AttributionContextTracing.scala:46)
	at com.databricks.logging.AttributionContextTracing.withAttributionContext$(AttributionContextTracing.scala:43)
	at com.databricks.threading.ContextBoundRunnable.withAttributionContext(ContextBoundRunnable.scala:6)
	at com.databricks.threading.ContextBoundRunnable.$anonfun$run$1(ContextBoundRunnable.scala:15)
	at scala.runtime.java8.JFunction0$mcV$sp.apply(JFunction0$mcV$sp.java:23)
	at com.databricks.context.integrity.IntegrityCheckContext$ThreadLocalStorage$.withValue(IntegrityCheckContext.scala:73)
	at com.databricks.threading.ContextBoundRunnable.run(ContextBoundRunnable.scala:14)
	at com.databricks.threading.DatabricksExecutionContext$InstrumentedRunnable.run(DatabricksExecutionContext.scala:36)
	at com.databricks.threading.NamedExecutor$$anon$2.run(NamedExecutor.scala:477)
	at java.base/java.util.concurrent.ThreadPoolExecutor.runWorker(ThreadPoolExecutor.java:1128)
	at java.base/java.util.concurrent.ThreadPoolExecutor$Worker.run(ThreadPoolExecutor.java:628)
	at java.base/java.lang.Thread.run(Thread.java:829)

In [0]:
test_df = steam_json.withColumn('test', F.explode('data'))
test_df.printSchema()
test_df.show()

---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-1910755200294727>:1
----> 1 test_df = steam_json.withColumn('test', F.explode('data'))
      2 test_df.printSchema()
      3 test_df.show()

File /databricks/spark/python/pyspark/instrumentation_utils.py:48, in _wrap_function.<locals>.wrapper(*args, **kwargs)
     46 start = time.perf_counter()
     47 try:
---> 48     res = func(*args, **kwargs)
     49     logger.log_success(
     50         module_name, class_name, function_name, time.perf_counter() - start, signature
     51     )
     52     return res

File /databricks/spark/python/pyspark/sql/dataframe.py:4758, in DataFrame.withColumn(self, colName, col)
   4753 if not isinstance(col, Column):
   4754     raise PySparkTypeError(
   4755         error_class="NOT_A_COLUMN",
   4756         message_parameters={"arg_name": "col", "arg_type": type(col).__name__},
   4757

In [0]:
steam_json.select("data").show(1)

+--------------------+
|                data|
+--------------------+
|{10, [Multi-playe...|
+--------------------+
only showing top 1 row



In [0]:
steam_rdd.printSchema()

root
 |-- data: struct (nullable = true)
 |    |-- appid: long (nullable = true)
 |    |-- categories: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- ccu: long (nullable = true)
 |    |-- developer: string (nullable = true)
 |    |-- discount: string (nullable = true)
 |    |-- genre: string (nullable = true)
 |    |-- header_image: string (nullable = true)
 |    |-- initialprice: string (nullable = true)
 |    |-- languages: string (nullable = true)
 |    |-- name: string (nullable = true)
 |    |-- negative: long (nullable = true)
 |    |-- owners: string (nullable = true)
 |    |-- platforms: struct (nullable = true)
 |    |    |-- linux: boolean (nullable = true)
 |    |    |-- mac: boolean (nullable = true)
 |    |    |-- windows: boolean (nullable = true)
 |    |-- positive: long (nullable = true)
 |    |-- price: string (nullable = true)
 |    |-- publisher: string (nullable = true)
 |    |-- release_date: string (nullable = true)
 |    |-

In [0]:
users = [
    {'id': 1, 'name': 'George', 'orders': [
        {'id': 1, 'value': 55.1},
        {'id': 2, 'value': 78.31},
        {'id': 4, 'value': 52.13}
    ]},
    {'id': 2, 'name': 'Hughes', 'orders': [
        {'id': 3, 'value': 31.19},
        {'id': 5, 'value': 131.1}
    ]}
]
users_rdd = sc.parallelize(users)

schema = StructType([
    StructField('id', IntegerType(), True),
    StructField('name', StringType(), True),
    StructField('orders', ArrayType(
        StructType([
            StructField('id', IntegerType(), True),
            StructField('value', FloatType(), True)
        ])
    ), True)
])

users_df = spark.createDataFrame(users_rdd, schema=schema)
users_df.printSchema()
users_df.show()

Out[14]: pyspark.rdd.RDD